### Introduction to Machine Learning
##### https://keras.io/getting-started/sequential-model-guide/
#### prepared by Chao-Lin Liu
###### 目前這一個程式的辨識率很差
###### 改變一下你的 sequential model 的設定，看看能不能達成比較高的辨識率？
###### 能不能把辨識率提高到 90% 以上？

In [29]:
import tensorflow as tf

In [30]:
import pandas as pd
import numpy as np
df = pd.read_csv("practice16.train.csv")

In [31]:
df[0:5]

,f0,f1,f2,class
0,0.633209,0.617612,0.163622,1
1,0.812224,0.241355,0.069014,1
2,0.638819,0.938740,0.834770,2
3,0.600864,0.603507,0.616036,2
4,0.075439,0.852837,0.337803,1


In [32]:
X_train = np.array(df.iloc[:,0:3])
y_train = np.array(df.iloc[:,3:4]).reshape(-1)

In [33]:
X_train[0:5]

array([[0.63320928, 0.61761157, 0.16362248],
       [0.81222375, 0.24135473, 0.06901366],
       [0.63881877, 0.93874007, 0.83477015],
       [0.60086396, 0.60350725, 0.61603573],
       [0.07543909, 0.85283652, 0.33780251]])

In [34]:
#shuffle data
from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train, y_train)

In [35]:
# 以下的步驟在 Introduction to Machine learning 課程並沒有講到
# 我們把 y 的數值變成了向量 － one-hot vector
#
from sklearn.preprocessing import LabelEncoder
# from keras.utils import np_utils
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(y_train)
encoded_npy = encoder.transform(y_train)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_npy = tf.keras.utils.to_categorical(encoded_npy)

In [36]:
y_train[0:5]

array([1, 2, 3, 1, 1], dtype=int64)

In [37]:
dummy_npy[0:5]

array([[0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.]], dtype=float32)

In [40]:
# 你可以試著增加 Dense 層的數目、或者改變個別 Dense 層的 nodes 數目、或者改變 activation functions
#
# from keras.models import Sequential
# from keras.layers import Dense
# define baseline model
def baseline_model():
    # create model
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(5, input_dim=3, activation='relu'))
    model.add(tf.keras.layers.Dense(5, activation='relu'))
    model.add(tf.keras.layers.Dense(4, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [41]:
model2 = baseline_model()

In [42]:
len(X_train)

3000

In [43]:
y_train[0:5]

array([1, 2, 3, 1, 1], dtype=int64)

In [56]:
# 在這裡，你可以嘗試增減 epochs 和 batch_size
model2.fit(X_train , dummy_npy, epochs=500, batch_size=200, verbose=1, shuffle=True)

Epoch 1/500
15/15 [==============================] - 0s 4ms/step - loss: 0.3590 - accuracy: 0.9330
Epoch 2/500
15/15 [==============================] - 0s 3ms/step - loss: 0.3563 - accuracy: 0.9343
Epoch 3/500
15/15 [==============================] - 0s 3ms/step - loss: 0.3537 - accuracy: 0.9337
Epoch 4/500
15/15 [==============================] - 0s 3ms/step - loss: 0.3511 - accuracy: 0.9337
Epoch 5/500
15/15 [==============================] - 0s 3ms/step - loss: 0.3484 - accuracy: 0.9357
Epoch 6/500
15/15 [==============================] - 0s 3ms/step - loss: 0.3459 - accuracy: 0.9357
Epoch 7/500
15/15 [==============================] - 0s 4ms/step - loss: 0.3433 - accuracy: 0.9363
Epoch 8/500
15/15 [==============================] - 0s 3ms/step - loss: 0.3408 - accuracy: 0.9367
Epoch 9/500
15/15 [==============================] - 0s 3ms/step - loss: 0.3384 - accuracy: 0.9367
Epoch 10/500
15/15 [==============================] - 0s 3ms/step - loss: 0.3359 - accuracy: 0.9370
Epoch 11/

15/15 [==============================] - 0s 3ms/step - loss: 0.2024 - accuracy: 0.9557
Epoch 84/500
15/15 [==============================] - 0s 3ms/step - loss: 0.2012 - accuracy: 0.9553
Epoch 85/500
15/15 [==============================] - 0s 3ms/step - loss: 0.1999 - accuracy: 0.9567
Epoch 86/500
15/15 [==============================] - 0s 3ms/step - loss: 0.1987 - accuracy: 0.9563
Epoch 87/500
15/15 [==============================] - 0s 3ms/step - loss: 0.1975 - accuracy: 0.9567
Epoch 88/500
15/15 [==============================] - 0s 3ms/step - loss: 0.1964 - accuracy: 0.9563
Epoch 89/500
15/15 [==============================] - 0s 3ms/step - loss: 0.1951 - accuracy: 0.9563
Epoch 90/500
15/15 [==============================] - 0s 3ms/step - loss: 0.1940 - accuracy: 0.9580
Epoch 91/500
15/15 [==============================] - 0s 3ms/step - loss: 0.1929 - accuracy: 0.9557
Epoch 92/500
15/15 [==============================] - 0s 3ms/step - loss: 0.1918 - accuracy: 0.9563
Epoch 93/500


15/15 [==============================] - 0s 3ms/step - loss: 0.1363 - accuracy: 0.9687
Epoch 165/500
15/15 [==============================] - 0s 3ms/step - loss: 0.1357 - accuracy: 0.9643
Epoch 166/500
15/15 [==============================] - 0s 3ms/step - loss: 0.1346 - accuracy: 0.9707
Epoch 167/500
15/15 [==============================] - 0s 4ms/step - loss: 0.1343 - accuracy: 0.9657
Epoch 168/500
15/15 [==============================] - 0s 3ms/step - loss: 0.1334 - accuracy: 0.9703
Epoch 169/500
15/15 [==============================] - 0s 3ms/step - loss: 0.1325 - accuracy: 0.9677
Epoch 170/500
15/15 [==============================] - 0s 3ms/step - loss: 0.1320 - accuracy: 0.9680
Epoch 171/500
15/15 [==============================] - 0s 4ms/step - loss: 0.1313 - accuracy: 0.9677
Epoch 172/500
15/15 [==============================] - 0s 3ms/step - loss: 0.1306 - accuracy: 0.9680
Epoch 173/500
15/15 [==============================] - 0s 3ms/step - loss: 0.1300 - accuracy: 0.9677
Epoc

15/15 [==============================] - 0s 3ms/step - loss: 0.0771 - accuracy: 0.9860
Epoch 246/500
15/15 [==============================] - 0s 3ms/step - loss: 0.0766 - accuracy: 0.9850
Epoch 247/500
15/15 [==============================] - 0s 3ms/step - loss: 0.0760 - accuracy: 0.9873
Epoch 248/500
15/15 [==============================] - 0s 5ms/step - loss: 0.0758 - accuracy: 0.9850
Epoch 249/500
15/15 [==============================] - 0s 4ms/step - loss: 0.0752 - accuracy: 0.9870
Epoch 250/500
15/15 [==============================] - 0s 3ms/step - loss: 0.0745 - accuracy: 0.9863
Epoch 251/500
15/15 [==============================] - 0s 3ms/step - loss: 0.0742 - accuracy: 0.9870
Epoch 252/500
15/15 [==============================] - 0s 3ms/step - loss: 0.0736 - accuracy: 0.9863
Epoch 253/500
15/15 [==============================] - 0s 3ms/step - loss: 0.0731 - accuracy: 0.9877
Epoch 254/500
15/15 [==============================] - 0s 3ms/step - loss: 0.0728 - accuracy: 0.9877
Epoc

15/15 [==============================] - 0s 3ms/step - loss: 0.0531 - accuracy: 0.9930
Epoch 327/500
15/15 [==============================] - 0s 3ms/step - loss: 0.0530 - accuracy: 0.9913
Epoch 328/500
15/15 [==============================] - 0s 4ms/step - loss: 0.0530 - accuracy: 0.9910
Epoch 329/500
15/15 [==============================] - 0s 4ms/step - loss: 0.0527 - accuracy: 0.9920
Epoch 330/500
15/15 [==============================] - 0s 3ms/step - loss: 0.0526 - accuracy: 0.9890
Epoch 331/500
15/15 [==============================] - 0s 3ms/step - loss: 0.0523 - accuracy: 0.9923
Epoch 332/500
15/15 [==============================] - 0s 3ms/step - loss: 0.0520 - accuracy: 0.9913
Epoch 333/500
15/15 [==============================] - 0s 3ms/step - loss: 0.0520 - accuracy: 0.9927
Epoch 334/500
15/15 [==============================] - 0s 3ms/step - loss: 0.0517 - accuracy: 0.9913
Epoch 335/500
15/15 [==============================] - 0s 3ms/step - loss: 0.0516 - accuracy: 0.9930
Epoc

15/15 [==============================] - 0s 3ms/step - loss: 0.0425 - accuracy: 0.9930
Epoch 408/500
15/15 [==============================] - 0s 3ms/step - loss: 0.0426 - accuracy: 0.9930
Epoch 409/500
15/15 [==============================] - 0s 3ms/step - loss: 0.0425 - accuracy: 0.9913
Epoch 410/500
15/15 [==============================] - 0s 3ms/step - loss: 0.0424 - accuracy: 0.9910
Epoch 411/500
15/15 [==============================] - 0s 3ms/step - loss: 0.0424 - accuracy: 0.9913
Epoch 412/500
15/15 [==============================] - 0s 3ms/step - loss: 0.0421 - accuracy: 0.9950
Epoch 413/500
15/15 [==============================] - 0s 3ms/step - loss: 0.0419 - accuracy: 0.9940
Epoch 414/500
15/15 [==============================] - 0s 3ms/step - loss: 0.0421 - accuracy: 0.9937
Epoch 415/500
15/15 [==============================] - 0s 3ms/step - loss: 0.0419 - accuracy: 0.9933
Epoch 416/500
15/15 [==============================] - 0s 3ms/step - loss: 0.0416 - accuracy: 0.9943
Epoc

15/15 [==============================] - 0s 3ms/step - loss: 0.0361 - accuracy: 0.9947
Epoch 489/500
15/15 [==============================] - 0s 3ms/step - loss: 0.0358 - accuracy: 0.9950
Epoch 490/500
15/15 [==============================] - 0s 3ms/step - loss: 0.0365 - accuracy: 0.9933
Epoch 491/500
15/15 [==============================] - 0s 3ms/step - loss: 0.0359 - accuracy: 0.9940
Epoch 492/500
15/15 [==============================] - 0s 3ms/step - loss: 0.0356 - accuracy: 0.9953
Epoch 493/500
15/15 [==============================] - 0s 3ms/step - loss: 0.0358 - accuracy: 0.9947
Epoch 494/500
15/15 [==============================] - 0s 3ms/step - loss: 0.0355 - accuracy: 0.9957
Epoch 495/500
15/15 [==============================] - 0s 3ms/step - loss: 0.0355 - accuracy: 0.9953
Epoch 496/500
15/15 [==============================] - 0s 3ms/step - loss: 0.0354 - accuracy: 0.9963
Epoch 497/500
15/15 [==============================] - 0s 3ms/step - loss: 0.0354 - accuracy: 0.9953
Epoc

In [45]:
df2 = pd.read_csv("practice16.test.csv")

In [46]:
testX = np.array(df2.iloc[:,0:3])

In [47]:
predicty = model2.predict(testX)

In [48]:
predicty[0:5]

array([[0.01409848, 0.9698454 , 0.01375962, 0.00229652],
       [0.02413806, 0.60913044, 0.33573282, 0.03099868],
       [0.0056949 , 0.11058283, 0.84254307, 0.0411793 ],
       [0.01027375, 0.66000813, 0.31203344, 0.01768462],
       [0.02223637, 0.9395943 , 0.03245646, 0.00571283]], dtype=float32)

In [49]:
testy = np.array(df2.iloc[:,3:4]).reshape(-1)

In [50]:
predicty = encoder.inverse_transform([np.argmax(item) for item in predicty])

In [51]:
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(testy,predicty))

[[  0  10   0   0]
 [  0 238  11   0]
 [  0   1 229   0]
 [  0   0  11   0]]


In [52]:
print(classification_report(testy,predicty))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        10
           1       0.96      0.96      0.96       249
           2       0.91      1.00      0.95       230
           3       0.00      0.00      0.00        11

    accuracy                           0.93       500
   macro avg       0.47      0.49      0.48       500
weighted avg       0.90      0.93      0.91       500



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [53]:
predicty[0:5]

array([1, 1, 2, 1, 1], dtype=int64)

In [54]:
len(predicty)

500

In [55]:
togo = open('practice16.output.csv', "w")
togo.write('id,Predicted\n')
for index in range(0,len(predicty)):
    togo.write(str(index+1)+","+str(predicty[index])+'\n')
togo.close()